In [8]:
import numpy as np
import pandas as pd
import tensorflow
tensorflow.keras.__version__
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [54]:
#Creating Dataset and including the first row by setting no header as input
dataset = pd.read_csv('DAUTOSAAR.csv', header = None)
#Renaming the columns
dataset.columns = ['DATE', 'AutoSales', '3Mslope', '6Mslope', 'Recession']
#dataset.set_index('DATE')
print('Shape of the dataset: ' + str(dataset.shape))
print(dataset.head(20))

Shape of the dataset: (634, 5)
       DATE  AutoSales   3Mslope   6Mslope  Recession
0    1/1/67      7.837  0.000000  0.000000          0
1    2/1/67      6.850  0.000000  0.000000          0
2    3/1/67      7.220  0.000000  0.000000          0
3    4/1/67      8.164  0.004479  0.000000          0
4    5/1/67      7.961  0.014360  0.000000          0
5    6/1/67      8.186  0.008840  0.005709          0
6    7/1/67      7.939 -0.001461  0.007683          0
7    8/1/67      7.295 -0.007335 -0.000062          0
8    9/1/67      7.764 -0.006199 -0.003958          0
9   10/1/67      7.052 -0.007112 -0.006009          0
10  11/1/67      6.766 -0.007478 -0.008659          0
11  12/1/67      7.780 -0.000802 -0.002905          0
12   1/1/68      8.526  0.017730  0.005541          0
13   2/1/68      8.035  0.014760  0.006346          0
14   3/1/68      8.743  0.007838  0.012170          0
15   4/1/68      8.153 -0.001470  0.008757          0
16   5/1/68      8.694  0.004518  0.003900         

In [23]:
#Creating the dependent variable class
X = dataset.drop(["Recession" and "DATE"], axis=1)
y = dataset["Recession"]
print(X.shape, y.shape)

(634, 4) (634,)


In [24]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1, stratify=y)

In [25]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

/Users/chrismiller/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [28]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [29]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [35]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=4))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [36]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [37]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 100)               500       
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 202       
Total params: 10,802
Trainable params: 10,802
Non-trainable params: 0
_________________________________________________________________


In [38]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=20,
    shuffle=True,
    verbose=2
)

Train on 475 samples
Epoch 1/20
475/475 - 1s - loss: 0.5404 - accuracy: 0.8589
Epoch 2/20
475/475 - 0s - loss: 0.3090 - accuracy: 0.9053
Epoch 3/20
475/475 - 0s - loss: 0.1392 - accuracy: 0.9916
Epoch 4/20
475/475 - 0s - loss: 0.0488 - accuracy: 1.0000
Epoch 5/20
475/475 - 0s - loss: 0.0187 - accuracy: 1.0000
Epoch 6/20
475/475 - 0s - loss: 0.0095 - accuracy: 1.0000
Epoch 7/20
475/475 - 0s - loss: 0.0058 - accuracy: 1.0000
Epoch 8/20
475/475 - 0s - loss: 0.0039 - accuracy: 1.0000
Epoch 9/20
475/475 - 0s - loss: 0.0027 - accuracy: 1.0000
Epoch 10/20
475/475 - 0s - loss: 0.0019 - accuracy: 1.0000
Epoch 11/20
475/475 - 0s - loss: 0.0013 - accuracy: 1.0000
Epoch 12/20
475/475 - 0s - loss: 9.4244e-04 - accuracy: 1.0000
Epoch 13/20
475/475 - 0s - loss: 7.0246e-04 - accuracy: 1.0000
Epoch 14/20
475/475 - 0s - loss: 5.4077e-04 - accuracy: 1.0000
Epoch 15/20
475/475 - 0s - loss: 4.2462e-04 - accuracy: 1.0000
Epoch 16/20
475/475 - 0s - loss: 3.4471e-04 - accuracy: 1.0000
Epoch 17/20
475/475 - 0s

In [39]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

159/1 - 0s - loss: 1.4102e-04 - accuracy: 1.0000
Normal Neural Network - Loss: 0.00013646898077692508, Accuracy: 1.0


In [71]:
# MAKE PREDICTIONS
encoded_predictions = model.predict_classes(X_test_scaled[:634])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [72]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:100])}")

Predicted classes: [0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 1 1 0 0 1 0 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 1 0 0]
Actual Labels: [0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]


In [73]:
df = pd.DataFrame(prediction_labels) 
df2 = pd.DataFrame(y_test[:100])
df
# df3 = df.merge(df2, left_on='index', right_on='index')
# df3

,0
0,0
1,0
2,0
3,1
4,1
5,0
6,1
7,0
8,0
9,0


In [47]:
>>> df = pd.DataFrame([[0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,]], [0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
...                   columns=['Predicted', 'actual'])
>>> ax1 = df.plot.scatter(x='predicted',
...                       y='actual',
...                       c='DarkBlue')

AssertionError: 2 columns passed, passed data had 50 columns